In [27]:
import pandas_profiling as pp
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import seaborn as sns
import xgboost as xg
from lightgbm import LGBMRegressor
from matplotlib import pyplot as plt

In [28]:
def get_df(path,isTrain=True):
  df = pd.read_csv(path,index_col="id").drop(columns=["RID"])
  df["placement date"] = pd.to_datetime(df["placement date"])
  df["delivery date"] = pd.to_datetime(df["delivery date"])
  df["days"] = (df["delivery date"] - df["placement date"]).dt.days
  df["address code"] = (df["address code"]).astype(int)

  if isTrain:
    df["cost"] = df["price"]*(1-df["discount"]) - df["profit"]
    df["pp"] = df["profit"]/df["price"]
  return df

In [29]:
df = get_df("trainf.csv")

In [30]:
df=df[((df['profit'])<=580.5394) & ((df['profit'])>=-312.0614)]

In [31]:
print(df.nunique())
print(df.info())

departure city       469
discount              12
price               4319
no of items           14
location               4
class                  3
segment                3
sub-class             17
delivery type          4
profit              5157
delivery date         37
address code         562
departure state       47
placement date        37
Profit percent       265
Discount percent    2576
days                   4
cost                5311
pp                   545
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6568 entries, 1 to 6700
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   departure city    6568 non-null   object        
 1   discount          6568 non-null   float64       
 2   price             6568 non-null   float64       
 3   no of items       6568 non-null   int64         
 4   location          6568 non-null   object        
 5   class             6568

In [32]:
def get_x(df):
  x_cols = ["discount","address code","days","no of items","segment","location","class","sub-class","delivery type"]
  ret = df[x_cols]
  for i in ret.select_dtypes(include=['object']).columns:
    ret = pd.concat([ret,pd.get_dummies(ret[i],prefix=i)],axis=1)
  ret =  ret.select_dtypes('number')
  return ret


In [33]:
x_main = get_x(df)
y_main = df["pp"]
x_train, x_test,y_train, y_test  = train_test_split(x_main,y_main, test_size=0.2,random_state=20)

In [34]:
pip install catboost

In [35]:
from catboost import CatBoostRegressor
#random_search.best_params_
#from catboost import CatBoostRegressor
X = get_x(df)
Y = df["pp"]
model=CatBoostRegressor(bagging_temperature=5,
 depth=4,
 l2_leaf_reg=1,
 learning_rate=0.2,
 random_strength =7)
model.fit(X,Y)

0:	learn: 0.3833325	total: 2.01ms	remaining: 2.01s
1:	learn: 0.3329342	total: 3.26ms	remaining: 1.63s
2:	learn: 0.2886951	total: 4.21ms	remaining: 1.4s
3:	learn: 0.2581266	total: 5.13ms	remaining: 1.28s
4:	learn: 0.2372989	total: 6.07ms	remaining: 1.21s
5:	learn: 0.2165490	total: 7.04ms	remaining: 1.17s
6:	learn: 0.1994012	total: 8.18ms	remaining: 1.16s
7:	learn: 0.1863241	total: 9.11ms	remaining: 1.13s
8:	learn: 0.1775110	total: 10.2ms	remaining: 1.13s
9:	learn: 0.1678019	total: 11.2ms	remaining: 1.11s
10:	learn: 0.1583860	total: 12.1ms	remaining: 1.09s
11:	learn: 0.1478173	total: 13.3ms	remaining: 1.09s
12:	learn: 0.1423876	total: 14.2ms	remaining: 1.08s
13:	learn: 0.1387700	total: 15.1ms	remaining: 1.06s
14:	learn: 0.1353314	total: 16ms	remaining: 1.05s
15:	learn: 0.1330936	total: 19.5ms	remaining: 1.2s
16:	learn: 0.1300751	total: 20.8ms	remaining: 1.21s
17:	learn: 0.1276037	total: 22.2ms	remaining: 1.21s
18:	learn: 0.1257628	total: 23.6ms	remaining: 1.22s
19:	learn: 0.1246352	total

In [36]:
# X = get_x(df)
# Y = df["pp"]
# from catboost import CatBoostRegressor
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# def timer(start_time=None):
#     if not start_time:
#         start_time = datetime.now()
#         return start_time
#     elif start_time:
#         thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
#         tmin, tsec = divmod(temp_sec, 60)
#         print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))
# model=CatBoostRegressor()
# params={
#  "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
#  "depth"        : [ 4, 5, 6, 7, 8, 9, 10],
#  "l2_leaf_reg" : [ 1, 3, 5, 7, 12, 15 ],
#  "random_strength"            : [ 2, 3, 5, 7, 9, 12, 17 ],
#  "bagging_temperature" : [ 3, 4, 5 , 7 ]
    
# }
# random_search=RandomizedSearchCV(model,param_distributions=params,n_iter=100,scoring='r2',n_jobs=4,cv=2,verbose=1)
# from datetime import datetime
# # Here we go
# start_time = timer(None) # timing starts from this poi
# random_search.fit(X,Y)
# timer(start_time)

In [37]:
dff = get_df("trainf.csv",isTrain=False)
X = get_x(dff)
y_pred = model.predict(X)*dff["price"]
r2_score(dff['profit'], y_pred)

0.9059408200719933

In [38]:
df = get_df("test.csv",isTrain=False)
X = get_x(df)
df["profit"] = model.predict(X)*df["price"]